# Error correction codes

In [1]:
import numpy as np

## Init

In [66]:
class ECC:
    HemmingDisguise = {
        3: (7, 4),
        4: (15, 11),
        5: (31, 26),
        6: (63, 57),
        7: (127, 120)
    }
    UnknownSymbol = '?'    


    def __init__(self, alphabet: list) -> None:
        alpSize = len(alphabet)
        k = 3
        while (2 ** ECC.HemmingDisguise[k][1] < alpSize):
            k += 1
        n, m = ECC.HemmingDisguise[k]
        self.__k__ = k
        alpDict = {}
        alpDictRev = {}
        for i in range(alpSize):
            letter = alphabet[i]
            bits = f'{i:b}'.rjust(m, "0")
            alpDict[letter] = bits
            alpDictRev[bits] = letter
        self.__m__, self.__n__ = m, n
        self.__alphabetDictionary__ = alpDict
        self.__alphabetDictionaryReversed__ = alpDictRev


    def StrToCharBitsList(self, word: str) -> list:
        l = len(word)
        res = [''] * l
        m = self.__m__
        us = ECC.UnknownSymbol
        for charIndex in range(l):
            c = word[charIndex]
            if (c == us):
                res[charIndex] += us * m
            else:
                res[charIndex] += self.__alphabetDictionary__[c]

        return res
    

    def CharBitsListToStr(self, charsBitsList: list) -> str:
        res = ''
        alpDictRev = self.__alphabetDictionaryReversed__
        keys = list(alpDictRev.keys())
        for c in charsBitsList:
            if (c not in keys):
                res += ECC.UnknownSymbol
            else:
                res += alpDictRev[c]
        return res


    def CharBitsListToHammingCodeList(self, charsBitsList: list) -> list:
        k = self.__k__
        n = self.__n__
        l = len(charsBitsList)
        res = [''] * l
        for charIndex in range(l):
            buffer = str(charsBitsList[charIndex])
            # ksBuffer = [0] * k
            # Adding zero Ks
            for kIndex in range(k):
                step = 2 ** kIndex
                buffer = buffer[: step - 1] + '0' + buffer[step - 1 :]
            # Calculate Ks from end
            for kIndex in range(k):
                s = 0
                step = 2 ** kIndex
                for j in range(step - 1, n, step * 2):
                    for i in range(j, j + step):
                        if i != step - 1 and buffer[i] == '1':
                            s += 1
                currentK = s & 1
                # ksBuffer[kIndex] = currentK
                buffer = buffer[: step - 1] + str(currentK) + buffer[step :]
            res[charIndex] = buffer

        return res


    def WordToHammingCodeList(self, word: str) -> list:
        return self.CharBitsListToHammingCodeList(self.StrToCharBitsList(word))


    def HammingCodeListToCharBitsList(self, hammingCodeList: list) -> list:
        l = len(hammingCodeList)
        res = [''] * l
        n = self.__n__
        ksIndices = [2 ** kNumber - 1 for kNumber in range(self.__k__)]
        for charIndex in range(l):
            buffer = str(list(hammingCodeList)[charIndex])
            resBuffer = ''
            for i in range(n):
                if not (i in ksIndices):
                    resBuffer += buffer[i]
            res[charIndex] = resBuffer
            
        return res


    def GetKsListByHammingCodeList(self, hammingCodeList: list) -> list:
        k = self.__k__
        n = self.__n__
        l = len(hammingCodeList)
        ks = []
        ksIndices = [2 ** kNumber - 1 for kNumber in range(k)]
        for charIndex in range(l):
            buffer = str(hammingCodeList[charIndex])
            ksBuffer = []
            for i in range(n):
                if i in ksIndices:
                    ksBuffer.append(int(buffer[i]))
            ks.append(ksBuffer)

        return ks
    

    def CalculateKsByHammingCodeList(self, hammingCodeList: list) -> list:
        return self.GetKsListByHammingCodeList(self.CharBitsListToHammingCodeList(self.HammingCodeListToCharBitsList(hammingCodeList)))


    def MakeErrorsInHammingCodeList(self, hammingCodeList: list, errorsCount: int = 1) -> list:
        res = list(hammingCodeList)
        l = len(hammingCodeList)
        n = self.__n__
        for _ in range(errorsCount):
            charIndex = np.random.randint(l)
            bitIndex = np.random.randint(n)
            buffer = res[charIndex]
            c = buffer[bitIndex]
            buffer = buffer[: bitIndex] + ('0' if c == '1' else '1') + buffer[bitIndex + 1 :]
            res[charIndex] = buffer
            print(f'Maked error at char {charIndex + 1}, bit {bitIndex + 1}')

        return res


    def FixAndReturnCharBitsList(self, hammingCodeList: list) -> list:
        ks = self.GetKsListByHammingCodeList(hammingCodeList)
        newKs = self.CalculateKsByHammingCodeList(hammingCodeList)
        l = len(hammingCodeList)
        k = self.__k__
        # Finding errors
        for charIndex in range(l):
            for kIndex in range(k):
                ks[charIndex][kIndex] ^= newKs[charIndex][kIndex]
        # Fixing errors
        for charIndex in range(l):
            errorBitNumber = 0
            for kIndex in range(k):
                if ks[charIndex][kIndex] == 1:
                    errorBitNumber += 2 ** kIndex
            if errorBitNumber == 0:
                continue
            buffer = hammingCodeList[charIndex]
            c = buffer[errorBitNumber - 1]
            buffer = buffer[: errorBitNumber - 1] + ('0' if c == '1' else '1') + buffer[errorBitNumber :]
            hammingCodeList[charIndex] = buffer
        
        return self.HammingCodeListToCharBitsList(hammingCodeList)
        

## Implementation

In [76]:
alp = [str(i) for i in range(10)]
word = '0123456789'
l = len(word)

ecc = ECC(alp)
charBits = ecc.StrToCharBitsList(word)
hammingCode = ecc.WordToHammingCodeList(word)
ks = ecc.GetKsListByHammingCodeList(hammingCode)

hammingCodeWithErrors = ecc.MakeErrorsInHammingCodeList(hammingCode)
hammingCodeWithErrors_reserved = list(hammingCodeWithErrors.copy())
errorWord = ecc.CharBitsListToStr(ecc.HammingCodeListToCharBitsList(hammingCodeWithErrors))

decoded = ecc.CharBitsListToStr(ecc.FixAndReturnCharBitsList(hammingCodeWithErrors))

print(f'Usual word: {word}')
print(f'Error word: {errorWord}')
print(f'Fixed word: {decoded}')
print(f'Hamming code: {hammingCode}')
print(f'Er-s hamming: {hammingCodeWithErrors_reserved}')

Maked error at char 2, bit 1
Usual word: 0123456789
Error word: 0123456789
Fixed word: 0123456789
Hamming code: ['0000000', '1101001', '0101010', '1000011', '1001100', '0100101', '1100110', '0001111', '1110000', '0011001']
Er-s hamming: ['0000000', '0101001', '0101010', '1000011', '1001100', '0100101', '1100110', '0001111', '1110000', '0011001']
